In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
import numpy as np
import pandas as pd

from collections import Counter

import torch
import torch.nn as nn

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [0]:
path='gdrive/My Drive/netflix_colab/data/'

***

# CREATING PYTORCH DATASET CLASS:

In [0]:
class Recommender_Data_Classifn(Dataset):

    def __init__(self, path, csv_file_name):
        
        df = pd.read_csv(path + csv_file_name, index_col = 0)
        x  = df.iloc[:, :-1].values
        y  = np.array([1 if i <3 else 0 for i in df.iloc[:, -1].values]) # POSITIVE & NEGATIVE
        self.len = df.shape[0]
        
        self.x_data = torch.from_numpy(x).type(torch.LongTensor)
        self.y_data = torch.from_numpy(y).type(torch.LongTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [0]:
train_dataset = Recommender_Data_Classifn(path, 'df_final_trainset.csv')
test_dataset  = Recommender_Data_Classifn(path, 'df_final_testset.csv')

In [0]:
train_dataset.x_data

tensor([[    0,     0],
        [    1,     1],
        [    2,     2],
        ...,
        [ 3652,  8903],
        [   56, 10623],
        [  263, 39396]])

In [0]:
train_dataset.y_data

tensor([0, 0, 0,  ..., 1, 0, 0])

***

# TRAIN-EVALUATION FUNCTION:

In [0]:
def fn_train_eval(model, train_set, test_set, batch_size, criterion, optimizer, n_epochs):
    
    # INNER FNS---------------------------------------------
    def retrn_scalar_y_pred(y_pred):
        _, y_pred_scalar = torch.max(y_pred, dim = 1) # idxs of max vals of each y_pred
        return y_pred_scalar

    def retrn_n_matches(y_pred_scalar, target):
        n_matches = (y_pred_scalar == target).sum().item()
        return n_matches
    
    # TRAIN & TEST SET BATCH GENERATORS----------------------
    train_iterator = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    test_iterator  = DataLoader(dataset=test_set,  batch_size=batch_size, shuffle=True, num_workers=2)
    
    # EPOCH LOOPS--------------------------------------------
    listO_models = []
    listO_train_losses, listO_test_losses = [], []
    listO_train_acc, listO_test_acc = [], []
    c1, c2 = 'print', 'print'
    for epoch in range(n_epochs):
        model.to(device)
        
        # TRAIN_LOOPS-----------------------------------------
        n_matches_train = 0
        iter_loss_train = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            
            movies  = Variable(batch[0][:, 0]).to(device)
            users   = Variable(batch[0][:, 1]).to(device)
            ratings = Variable(batch[1]).to(device)                               # make data diffrentiable
            
            optimizer.zero_grad()                                                 # Init
            y_pred = model(users, movies)                                         # Predict
            loss_train = criterion(y_pred, ratings)                               # Calculate loss
            iter_loss_train += loss_train.data.item()                             # Accumulate the loss
            loss_train.backward()                                                 # Backprop
            optimizer.step()                                                      # Update weights
            
            if c1 == 'print':
                print('TRAINING >>>>')                                            # confirmation!!
                c1 = 'no_print' 
            
            y_pred_scalar = retrn_scalar_y_pred(y_pred)
            n_matches_train += retrn_n_matches(y_pred_scalar, ratings)
       
        # COLLECT TRAIN ACCURACIES & LOSSES--------------------    
        train_acc = n_matches_train/len(train_set) 
        listO_train_acc.append(train_acc)
        train_loss = iter_loss_train/(batch_idx+1)
        listO_train_losses.append(train_loss)
        
        # COLLECT MODELS---------------------------------------
        listO_models.append(model)
        
        # EVAL_LOOPS-------------------------------------------
        n_matches_test = 0
        iter_loss_test = 0.0
        model.eval()
        with torch.no_grad():  
            for batch_idx, batch in enumerate(test_iterator):
                
                movies  = Variable(batch[0][:, 0]).to(device)
                users   = Variable(batch[0][:, 1]).to(device)
                ratings = Variable(batch[1]).to(device)  
                
                y_pred = model(users, movies)                         # Predict
                loss_test = criterion(y_pred, ratings)                # Calculate loss
                iter_loss_test += loss_test.data.item() 
                
                if c2 == 'print':
                    print('EVALUATING >>>>')                          # confirmation!!
                    c2 = 'no_print' 
                
                y_pred_scalar = retrn_scalar_y_pred(y_pred)
                n_matches_test += retrn_n_matches(y_pred_scalar, ratings)
        
        # COLLECT TEST ACCURACIES & LOSSES---------------------
        test_acc = n_matches_test/len(test_set)
        listO_test_acc.append(test_acc)
        test_loss = iter_loss_test/(batch_idx+1)
        listO_test_losses.append(test_loss)

        # DISPLAY-----------------------------------------------
        params = [epoch+1, loss_train, loss_test, train_acc, test_acc]
        print('Epoch: {:03}  train_loss: {:.3f}  |  test_loss: {:.3f}  |  train_acc: {:.4f}  |  test_acc: {:.4f}'.format(*params))
        
    print()
    print('RETURNED: listO_train_losses, listO_test_losses, listO_train_acc, listO_test_acc, listO_models')
    print()
    return listO_train_losses, listO_test_losses, listO_train_acc, listO_test_acc, listO_models

***

# MODEL:

In [0]:
class Recommender_Classify_2(torch.nn.Module):
    
    def __init__(self, n_users, n_items, embedding_dim, dropout1, hidden_dim, dropout2, n_classes):
        super().__init__()
        
	     # create user & item embeddings of same size:
        self.user_embeddings = nn.Embedding(n_users, embedding_dim, sparse=False)
        self.item_embeddings = nn.Embedding(n_items, embedding_dim, sparse=False)
        
        self.weights1 = nn.Linear(embedding_dim*2, hidden_dim)
        self.weights2 = nn.Linear(hidden_dim, n_classes)
        
        self.batch_norm1 = nn.BatchNorm1d(embedding_dim*2)
        self.batch_norm2 = nn.BatchNorm1d(hidden_dim)
        
        self.dropout1 = nn.Dropout2d(p=dropout1)
        self.dropout2 = nn.Dropout2d(p=dropout2)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self, user, item):
        
    	# CREATING USER LATENT VARIABLES
        
        # embedding
        user_vec = self.user_embeddings(user)
        item_vec = self.item_embeddings(item)
        
        # representation of iput(concat user & item vectors):
        input_vec = torch.cat((user_vec, item_vec), dim = 1)
                              
        # layer 1
        hidden = self.batch_norm1(input_vec) 
        hidden = self.dropout1(hidden) 
        hidden = self.weights1(hidden) 
        hidden = self.relu(hidden)
        # layers 2 
        hidden = self.batch_norm2(hidden) 
        hidden = self.dropout2(hidden) 
        hidden = self.weights2(hidden) 
        
        y_pred     = self.softmax(hidden)
        return y_pred

### TRIAL_1:

In [0]:
n_users, n_items, embedding_dim, hidden_dim, n_classes = 50000, 5000, 50, 50, 2
dropout1, dropout2 = 0.5, 0.25

model = Recommender_Classify_2(n_users, n_items, embedding_dim, dropout1, hidden_dim, dropout2, n_classes)

learning_rate = 1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

train_set = train_dataset
test_set = test_dataset
batch_size = 1000                                                  
n_epochs = 10

%time listO_train_losses, listO_test_losses, listO_train_acc, listO_test_acc, listO_models = fn_train_eval(model, train_set, test_set, batch_size, criterion, optimizer, n_epochs)

TRAINING >>>>
EVALUATING >>>>
Epoch: 001  train_loss: 0.448  |  test_loss: 0.483  |  train_acc: 0.8533  |  test_acc: 0.8541
Epoch: 002  train_loss: 0.500  |  test_loss: 0.441  |  train_acc: 0.8542  |  test_acc: 0.8541
Epoch: 003  train_loss: 0.446  |  test_loss: 0.459  |  train_acc: 0.8542  |  test_acc: 0.8541
Epoch: 004  train_loss: 0.470  |  test_loss: 0.477  |  train_acc: 0.8542  |  test_acc: 0.8541
Epoch: 005  train_loss: 0.482  |  test_loss: 0.462  |  train_acc: 0.8542  |  test_acc: 0.8541
Epoch: 006  train_loss: 0.463  |  test_loss: 0.474  |  train_acc: 0.8542  |  test_acc: 0.8541
Epoch: 007  train_loss: 0.475  |  test_loss: 0.459  |  train_acc: 0.8542  |  test_acc: 0.8541
Epoch: 008  train_loss: 0.441  |  test_loss: 0.477  |  train_acc: 0.8542  |  test_acc: 0.8541
Epoch: 009  train_loss: 0.451  |  test_loss: 0.477  |  train_acc: 0.8542  |  test_acc: 0.8541
Epoch: 010  train_loss: 0.451  |  test_loss: 0.465  |  train_acc: 0.8542  |  test_acc: 0.8541

RETURNED: listO_train_losses,

### TRIAL 2:

In [0]:
n_users, n_items, embedding_dim, hidden_dim, n_classes = 50000, 5000, 100, 100, 2
dropout1, dropout2 = 0, 0

model = Recommender_Classify_2(n_users, n_items, embedding_dim, dropout1, hidden_dim, dropout2, n_classes)

learning_rate = 1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

train_set = train_dataset
test_set = test_dataset
batch_size = 1000                                                  
n_epochs = 10

%time listO_train_losses, listO_test_losses, listO_train_acc, listO_test_acc, listO_models = fn_train_eval(model, train_set, test_set, batch_size, criterion, optimizer, n_epochs)

TRAINING >>>>
EVALUATING >>>>
Epoch: 001  train_loss: 0.442  |  test_loss: 0.438  |  train_acc: 0.8550  |  test_acc: 0.8588
Epoch: 002  train_loss: 0.460  |  test_loss: 0.453  |  train_acc: 0.8623  |  test_acc: 0.8615
Epoch: 003  train_loss: 0.428  |  test_loss: 0.462  |  train_acc: 0.8665  |  test_acc: 0.8614
Epoch: 004  train_loss: 0.412  |  test_loss: 0.443  |  train_acc: 0.8688  |  test_acc: 0.8619
Epoch: 005  train_loss: 0.443  |  test_loss: 0.464  |  train_acc: 0.8703  |  test_acc: 0.8616
Epoch: 006  train_loss: 0.454  |  test_loss: 0.422  |  train_acc: 0.8716  |  test_acc: 0.8605
Epoch: 007  train_loss: 0.468  |  test_loss: 0.475  |  train_acc: 0.8727  |  test_acc: 0.8615
Epoch: 008  train_loss: 0.418  |  test_loss: 0.451  |  train_acc: 0.8737  |  test_acc: 0.8609
Epoch: 009  train_loss: 0.461  |  test_loss: 0.433  |  train_acc: 0.8746  |  test_acc: 0.8606
Epoch: 010  train_loss: 0.451  |  test_loss: 0.458  |  train_acc: 0.8756  |  test_acc: 0.8611

RETURNED: listO_train_losses,

### TRIAL 3: (REDUCING LEARNING RATE & ADDING WEIGHT DECAY )

In [0]:
n_users, n_items, embedding_dim, hidden_dim, n_classes = 50000, 5000, 100, 100, 2
dropout1, dropout2 = 0, 0

model = Recommender_Classify_2(n_users, n_items, embedding_dim, dropout1, hidden_dim, dropout2, n_classes)

learning_rate = 1e-3
weight_decay = 1e-5

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

train_set = train_dataset
test_set = test_dataset
batch_size = 1000                                                  
n_epochs = 20

%time listO_train_losses, listO_test_losses, listO_train_acc, listO_test_acc, listO_models = fn_train_eval(model, train_set, test_set, batch_size, criterion, optimizer, n_epochs)

TRAINING >>>>
EVALUATING >>>>
Epoch: 001  train_loss: 0.465  |  test_loss: 0.455  |  train_acc: 0.8504  |  test_acc: 0.8552
Epoch: 002  train_loss: 0.456  |  test_loss: 0.439  |  train_acc: 0.8574  |  test_acc: 0.8597
Epoch: 003  train_loss: 0.455  |  test_loss: 0.439  |  train_acc: 0.8618  |  test_acc: 0.8614
Epoch: 004  train_loss: 0.431  |  test_loss: 0.453  |  train_acc: 0.8649  |  test_acc: 0.8623
Epoch: 005  train_loss: 0.442  |  test_loss: 0.446  |  train_acc: 0.8678  |  test_acc: 0.8625
Epoch: 006  train_loss: 0.466  |  test_loss: 0.451  |  train_acc: 0.8709  |  test_acc: 0.8627
Epoch: 007  train_loss: 0.420  |  test_loss: 0.457  |  train_acc: 0.8740  |  test_acc: 0.8626
Epoch: 008  train_loss: 0.437  |  test_loss: 0.426  |  train_acc: 0.8770  |  test_acc: 0.8625
Epoch: 009  train_loss: 0.428  |  test_loss: 0.463  |  train_acc: 0.8792  |  test_acc: 0.8615
Epoch: 010  train_loss: 0.431  |  test_loss: 0.422  |  train_acc: 0.8815  |  test_acc: 0.8609
Epoch: 011  train_loss: 0.447 

### BEST PERFORMANCE - TRIAL 3   - Epoch: 006  train_loss: 0.466  |  test_loss: 0.451  |  train_acc: 0.8709  |  test_acc: 0.8627